In [1]:
import nltk
from nltk.corpus import treebank
from nltk.corpus.reader.conll import *
import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
train_data = "/media/sf_communalflat/ihatelinguistics/unamb_sent_14_6.conllu" 
train_reader = ConllCorpusReader(root = '', fileids = [train_data], columntypes=['ignore', 'ignore', 'words', 'pos', 'chunk'])

In [3]:
sents = list(train_reader.iob_sents())
len(sents)*(3/4) 

28881.0

In [4]:
train_sents = sents[:28881]
test_sents = sents[28881: ]
print(len(train_sents))
print(len(test_sents))

28881
9627


In [5]:
X_train = [[word[0].lower()  for word in sent ] for sent in train_sents ]
X_test = [[word[0].lower()  for word in sent ] for sent in test_sents ]
Y_train = [[word[1]  for word in sent ] for sent in train_sents ]
Y_test = [[word[1]  for word in sent ] for sent in test_sents ]

#all words
words = list(set([word[0].lower() for sent in train_sents+test_sents for word in sent ]))

word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}

#all labels
labels = list(set([word[1] for sent in train_sents for word in sent ]))

label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}

maxlen = max([len(sent) for sent in train_sents+test_sents])


In [6]:
emb_path = '/media/sf_communalflat/ihatelinguistics/wiki.ru.vec'

words = []

embeddings_index = {}
f = open(emb_path)
for line in f:
    values = line.split()
    if len(values) == 301:
        word = values[0]
        words.append(word)
        coefs = np.asarray(values[1:50], dtype='float32')
        embeddings_index[word] = coefs
f.close()


In [7]:
def get_window(word_list, k):
    windows = []
    hk = int((k-1)/2)
    for i in range(len(word_list)):
        if i<hk:
            window=[]
            for ii in word_list[:i]:
                window.append(ii)
            for ii in word_list[i:(i+hk+1)]:
                window.append(ii)
            windows.append(window)
        elif i>(len(word_list)-(hk+1)):
            window=[]
            for ii in word_list[(i-hk):i]:
                window.append(ii)
            for ii in word_list[i:]:
                window.append(ii)
            windows.append(window)
        else:
            window=[]
            for ii in word_list[(i-hk):i]:
                window.append(ii)
            for ii in word_list[i:(i+hk+1)]:
                window.append(ii)
            windows.append(window)
    return windows


In [21]:
#values changed for embeddings len 49
def concatenate_embeddings_2(vector, k):
    a = np.zeros((int(k*49),), dtype='float32')
    for i in range(len(vector)):
        b = i*49
        c = b+49
        if embeddings_index.get(vector[i]) is not None:
            a[b:c]=embeddings_index.get(vector[i])
        else:
            a[b:c]=np.zeros((49,),dtype='float32')
    return a

In [22]:
X_train_window=[get_window(v, 3) for v in X_train[:300]]
X_train_enc=[[concatenate_embeddings_2(v,3) for v in x] for x in X_train_window]

X_test_window=[get_window(v, 3) for v in X_test[:100]]
X_test_enc=[[concatenate_embeddings_2(v,3) for v in x] for x in X_test_window]

In [10]:
max_label = max(label2ind.values())  + 1
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

Y_train_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in Y_train[:300]]
Y_train_enc = [[encode(c, max_label) for c in ey] for ey in Y_train_enc]

Y_test_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in Y_test[:100]]
Y_test_enc = [[encode(c, max_label) for c in ey] for ey in Y_test_enc]

In [36]:
print(maxlen)

230


In [31]:
X_train_pad = pad_sequences(X_train_enc, maxlen=maxlen)
Y_train_pad = pad_sequences(Y_train_enc, maxlen=maxlen)

X_test_pad = pad_sequences(X_test_enc, maxlen=maxlen)
Y_test_pad = pad_sequences(Y_test_enc, maxlen=maxlen)


In [32]:
print(X_train_pad.shape, Y_train_pad.shape, X_test_pad.shape, Y_test_pad.shape)

(300, 230, 147) (300, 230, 15) (100, 230, 147) (100, 230, 15)


In [27]:
max_features = len(word2ind)
embedding_size = 32
hidden_size = 32
out_size = len(label2ind)  + 1
nb_epoch = 5
batch_size = 32

In [28]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
#from keras.layers import Merge
#from keras.backend import tf


In [29]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Bidirectional(LSTM(hidden_size, return_sequences = True)))
model.add(TimeDistributed(Dense(out_size, activation = 'softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, Y_train_pad, batch_size=batch_size, epochs=nb_epoch, validation_split=0.1)

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (30, 230, 147)